# Action recognition using TAO ActionRecognitionNet

Transfer learning is the process of transferring learned features from one application to another. It is a commonly used training technique where you use a model trained on one task and re-train to use it on a different task. 

Train Adapt Optimize (TAO) Toolkit  is a simple and easy-to-use Python based AI toolkit for taking purpose-built AI models and customizing them with users' own data.

<img align="center" src="https://developer.nvidia.com/sites/default/files/akamai/embedded-transfer-learning-toolkit-software-stack-1200x670px.png" width="1080">


## Learning Objectives

In this notebook, you will learn how to leverage the simplicity and convenience of TAO to:

* Train 3D RGB only for action recognition on the subset of [HMDB51](https://serre-lab.clps.brown.edu/resource/hmdb-a-large-human-motion-database/) dataset.
* Evaluate the trained model.
* Run Inference on the trained model.
* Export the trained model to a .etlt file for deployment to DeepStream.

## Table of Contents

This notebook shows an example usecase of ActionRecognitionNet using Train Adapt Optimize (TAO) Toolkit.

0. [Set up env variables and map drives](#head-0)
1. [Installing the TAO launcher](#head-1)
2. [Prepare dataset and pre-trained model](#head-2)
3. [Provide training specification](#head-3)
4. [Run TAO training](#head-4)
5. [Evaluate trained models](#head-5)
6. [Inferences](#head-6)
7. [Deploy](#head-7)


## 0. Set up env variables and map drives <a class="anchor" id="head-0"></a>

When using the purpose-built pretrained models from NGC, please make sure to set the `$KEY` environment variable to the key as mentioned in the model overview. Failing to do so, can lead to errors when trying to load them as pretrained models.

The TAO launcher uses docker containers under the hood, and **for our data and results directory to be visible to the docker, they need to be mapped**. The launcher can be configured using the config file `~/.tao_mounts.json`. Apart from the mounts, you can also configure additional options like the Environment Variables and amount of Shared Memory available to the TAO launcher. <br>

`IMPORTANT NOTE:` The code below creates a sample `~/.tao_mounts.json`  file. Here, we can map directories in which we save the data, specs, results and cache. You should configure it for your specific case so these directories are correctly visible to the docker container.


In [1]:
%env HOST_DATA_DIR=/home/sandia/tao/data
# note: You could set the HOST_SPECS_DIR to folder of the experiments specs downloaded with the notebook
%env HOST_SPECS_DIR=/home/sandia/tao/specs
%env HOST_RESULTS_DIR=/home/sandia/tao/results

# Set your encryption key, and use the same key for all commands
%env KEY = nvidia_tao

env: HOST_DATA_DIR=/home/sandia/tao/data
env: HOST_SPECS_DIR=/home/sandia/tao/specs
env: HOST_RESULTS_DIR=/home/sandia/tao/results
env: KEY=nvidia_tao


In [3]:
! mkdir -p $HOST_DATA_DIR
! mkdir -p $HOST_SPECS_DIR
! mkdir -p $HOST_RESULTS_DIR

In [3]:
# Mapping up the local directories to the TAO docker.
import json
import os
mounts_file = os.path.expanduser("~/.tao_mounts.json")
tlt_configs = {
   "Mounts":[
       {
           "source": os.environ["HOST_DATA_DIR"],
           "destination": "/data"
       },
       {
           "source": os.environ["HOST_SPECS_DIR"],
           "destination": "/specs"
       },
       {
           "source": os.environ["HOST_RESULTS_DIR"],
           "destination": "/results"
       },
       {
           "source": os.path.expanduser("~/.cache"),
           "destination": "/root/.cache"
       }
   ],
   "DockerOptions": {
        "shm_size": "16G",
        "ulimits": {
            "memlock": -1,
            "stack": 67108864
         }
   }
}
# Writing the mounts file.
with open(mounts_file, "w") as mfile:
    json.dump(tlt_configs, mfile, indent=4)

In [5]:
!cat ~/.tao_mounts.json

{
    "Mounts": [
        {
            "source": "/home/sandia/tao/data",
            "destination": "/data"
        },
        {
            "source": "/home/sandia/tao/specs",
            "destination": "/specs"
        },
        {
            "source": "/home/sandia/tao/results",
            "destination": "/results"
        },
        {
            "source": "/home/sandia/.cache",
            "destination": "/root/.cache"
        }
    ],
    "DockerOptions": {
        "shm_size": "16G",
        "ulimits": {
            "memlock": -1,
            "stack": 67108864
        }
    }
}

## 1. Installing the TAO launcher <a class="anchor" id="head-1"></a>
The TAO launcher is a python package distributed as a python wheel listed in the `nvidia-pyindex` python index. You may install the launcher by executing the following cell.

Please note that TAO Toolkit recommends users to run the TAO launcher in a virtual env with python 3.6.9. You may follow the instruction in this [page](https://virtualenvwrapper.readthedocs.io/en/latest/install.html) to set up a python virtual env using the `virtualenv` and `virtualenvwrapper` packages. Once you have setup virtualenvwrapper, please set the version of python to be used in the virtual env by using the `VIRTUALENVWRAPPER_PYTHON` variable. You may do so by running

```sh
export VIRTUALENVWRAPPER_PYTHON=/path/to/bin/python3.x
```
where x >= 6 and <= 8

We recommend performing this step first and then launching the notebook from the virtual environment. In addition to installing TAO python package, please make sure of the following software requirements:
* python >=3.6.9 < 3.8.x
* docker-ce > 19.03.5
* docker-API 1.40
* nvidia-container-toolkit > 1.3.0-1
* nvidia-container-runtime > 3.4.0-1 (mine: 3.9.0)
* nvidia-docker2 > 2.5.0-1
* nvidia-driver > 455+

Once you have installed the pre-requisites, please log in to the docker registry nvcr.io by following the command below

```sh
docker login nvcr.io
```

You will be triggered to enter a username and password. The username is `$oauthtoken` and the password is the API key generated from `ngc.nvidia.com`. Please follow the instructions in the [NGC setup guide](https://docs.nvidia.com/ngc/ngc-overview/index.html#generating-api-key) to generate your own API key.

Please note that TAO Toolkit recommends users to run the TAO launcher in a virtual env with python >=3.6.9. You may follow the instruction in this [page](https://virtualenvwrapper.readthedocs.io/en/latest/install.html) to set up a python virtual env using the virtualenv and virtualenvwrapper packages.

In [1]:
# SKIP this step IF you have already installed the TAO launcher.
!pip3 install nvidia-pyindex
!pip3 install nvidia-tao

  Preparing metadata (setup.py) ... done
  Created wheel for nvidia-pyindex: filename=nvidia_pyindex-1.0.9-py3-none-any.whl size=8416 sha256=b1ff09957e7a314e5b876fb53cbb2706e76cfce6102151ec27ca309d58037ef7
  Stored in directory: /home/sandia/.cache/pip/wheels/1a/79/65/9cb980b5f481843cd9896e1579abc1c1f608b5f9e60ca90e03
Successfully built nvidia-pyindex
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 149 kB 8.1 MB/s            
     |████████████████████████████████| 127 kB 97.2 MB/s            
     |████████████████████████████████| 145 kB 88.5 MB/s            
     |████████████████████████████████| 58 kB 83.1 MB/s            
     |████████████████████████████████| 200 kB 86.9 MB/s            
     |████████████████████████████████| 156 kB 97.7 MB/s            
     |████████████████████████████████| 61 kB 34.9 MB/s             
     |████████████████████████████████| 133 kB 94.7 MB/s            
  Attempting uninstall:

In [16]:
# View the versions of the TAO launcher
!tao info

/bin/bash: tao: command not found


## 2. Prepare dataset and pre-trained model <a class="anchor" id="head-2"></a>

### 2.1 Prepare dataset

 We will be using the [HMDB51](https://serre-lab.clps.brown.edu/resource/hmdb-a-large-human-motion-database/) dataset for the tutorial. Download the HMDB51 dataset and unrar them firstly (We choose fall_floor/ride_bike for this tutorial): 

In [4]:
# install unrar
# NOTE: The following commands require `sudo`. You can run the command outside the notebook.
!sudo apt update
!sudo apt-get install unrar

[sudo] password for sandia: 
[sudo] password for sandia: 


In [6]:
# download the dataset and unrar the files
!wget -P $HOST_DATA_DIR http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar
!mkdir -p $HOST_DATA_DIR/videos && unrar x $HOST_DATA_DIR/hmdb51_org.rar $HOST_DATA_DIR/videos
!mkdir -p $HOST_DATA_DIR/raw_data
!unrar x $HOST_DATA_DIR/videos/fall_floor.rar $HOST_DATA_DIR/raw_data
!unrar x $HOST_DATA_DIR/videos/ride_bike.rar $HOST_DATA_DIR/raw_data

[sudo] password for sandia: 
mkdir: cannot create directory ‘/videos’: Permission denied
mkdir: cannot create directory ‘/raw_data’: Permission denied

UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal

Cannot open /videos/fall_floor.rar
No such file or directory
No files to extract

UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal

Cannot open /videos/ride_bike.rar
No such file or directory
No files to extract


Clone the dataset process script

In [ ]:
!git clone https://github.com/NVIDIA-AI-IOT/tao_toolkit_recipes

Install the dependency for data generator:

In [ ]:
!pip3 install xmltodict opencv-python

Run the process script. 

In [2]:
%cd ../../
!pwd

/home/sandia/tao
/home/sandia/tao


In [6]:
!cd tao_toolkit_recipes/tao_action_recognition/data_generation/ && bash ./preprocess_HMDB_RGB.sh $HOST_DATA_DIR/raw_data $HOST_DATA_DIR/processed_data

/home/sandia/tao/data/raw_data
/home/sandia/tao/data/processed_data
Preprocess pullup
f cnt: 82.0
f cnt: 82.0
f cnt: 82.0
f cnt: 71.0
f cnt: 81.0
f cnt: 81.0
f cnt: 80.0
f cnt: 80.0
f cnt: 80.0
f cnt: 80.0
f cnt: 80.0
f cnt: 100.0
f cnt: 80.0
f cnt: 80.0
f cnt: 80.0
f cnt: 81.0
f cnt: 81.0
f cnt: 81.0
f cnt: 81.0
f cnt: 80.0
f cnt: 81.0
f cnt: 79.0
f cnt: 79.0
f cnt: 79.0
f cnt: 82.0
f cnt: 82.0
f cnt: 82.0
f cnt: 82.0
f cnt: 82.0
f cnt: 82.0
f cnt: 80.0
f cnt: 79.0
f cnt: 80.0
f cnt: 81.0
f cnt: 81.0
f cnt: 81.0
f cnt: 79.0
f cnt: 79.0
f cnt: 80.0
f cnt: 80.0
f cnt: 80.0
f cnt: 80.0
f cnt: 80.0
f cnt: 80.0
f cnt: 80.0
f cnt: 80.0
f cnt: 79.0
f cnt: 88.0
f cnt: 88.0
f cnt: 88.0
f cnt: 80.0
f cnt: 79.0
f cnt: 80.0
f cnt: 84.0
f cnt: 84.0
f cnt: 84.0
f cnt: 82.0
f cnt: 82.0
f cnt: 82.0
f cnt: 80.0
f cnt: 79.0
f cnt: 80.0
f cnt: 82.0
f cnt: 82.0
f cnt: 56.0
f cnt: 82.0
f cnt: 82.0
f cnt: 68.0
f cnt: 114.0
f cnt: 84.0
f cnt: 82.0
f cnt: 80.0
f cnt: 43.0
f cnt: 80.0
f cnt: 84.0
f cnt: 106.0

In [3]:
# download the split files and unrar
!wget -P $HOST_DATA_DIR http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/test_train_splits.rar
!mkdir -p $HOST_DATA_DIR/splits && unrar x $HOST_DATA_DIR/test_train_splits.rar $HOST_DATA_DIR/splits

--2022-06-01 03:50:54--  http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/test_train_splits.rar
Resolving serre-lab.clps.brown.edu (serre-lab.clps.brown.edu)... 128.148.254.114
Connecting to serre-lab.clps.brown.edu (serre-lab.clps.brown.edu)|128.148.254.114|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/test_train_splits.rar [following]
--2022-06-01 03:50:55--  https://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/test_train_splits.rar
Connecting to serre-lab.clps.brown.edu (serre-lab.clps.brown.edu)|128.148.254.114|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 199521 (195K)
Saving to: ‘/home/sandia/tao/data/test_train_splits.rar’

test_train_splits.r 100%[===================>] 194.84K   190KB/s    in 1.0s    

2022-06-01 03:50:57 (190 KB/s) - ‘/home/sandia/tao/data/test_train_splits.rar’ saved [199521/199521]


UNRAR 5.50 freeware      Copyright (c) 1

In [6]:
# run split_HMDB to generate training split
!cd tao_toolkit_recipes/tao_action_recognition/data_generation/ && python3 ./split_dataset.py $HOST_DATA_DIR/processed_data $HOST_DATA_DIR/splits/testTrainMulti_7030_splits $HOST_DATA_DIR/train  $HOST_DATA_DIR/test

Traceback (most recent call last):
  File "./split_dataset.py", line 31, in <module>
    os.makedirs(target_train_path)
  File "/usr/lib/python3.6/os.py", line 220, in makedirs
    mkdir(name, mode)
PermissionError: [Errno 13] Permission denied: '/train'


In [8]:
# verify
!ls -l $HOST_DATA_DIR/train
!ls -l $HOST_DATA_DIR/train/ride_bike
!ls -l $HOST_DATA_DIR/test
!ls -l $HOST_DATA_DIR/test/ride_bike

total 36
drwxrwxr-x 72 sandia sandia 12288  6월  1 03:51 pullup
drwxrwxr-x 72 sandia sandia 12288  6월  1 03:51 pushup
drwxrwxr-x 72 sandia sandia 12288  6월  1 03:51 situp
ls: cannot access '/home/sandia/tao/data/train/ride_bike': No such file or directory
total 12
drwxrwxr-x 32 sandia sandia 4096  6월  1 03:51 pullup
drwxrwxr-x 32 sandia sandia 4096  6월  1 03:51 pushup
drwxrwxr-x 32 sandia sandia 4096  6월  1 03:51 situp
ls: cannot access '/home/sandia/tao/data/test/ride_bike': No such file or directory


We also provide scripts to preprocess [SHAD](https://best.sjtu.edu.cn/Data/View/990) dataset. The following cells for processing SHAD dataset is `Optional`.

`OPTIONAL:` Download the app based on NVOF SDK to generate optical flow. It is packaged with this notebook.

In [ ]:
#!echo <passwd> | sudo -S apt install -y libfreeimage-dev

In [ ]:
# # create train and test dir for raw videos and labels
# !mkdir -p $HOST_DATA_DIR/train_raw && mkdir -p $HOST_DATA_DIR/test_raw
# # download the dataset and unrar the files
# !wget -P $HOST_DATA_DIR https://best.sjtu.edu.cn/Assets/userfiles/sys_eb538c1c-65ff-4e82-8e6a-a1ef01127fed/files/ZIP/Bend-train.rar
# !unrar x $HOST_DATA_DIR/Bend-train.rar $HOST_DATA_DIR/train_raw
# !wget -P $HOST_DATA_DIR https://best.sjtu.edu.cn/Assets/userfiles/sys_eb538c1c-65ff-4e82-8e6a-a1ef01127fed/files/ZIP/Bend-test.rar
# !unrar x $HOST_DATA_DIR/Bend-test.rar $HOST_DATA_DIR/test_raw
# !wget -P $HOST_DATA_DIR https://best.sjtu.edu.cn/Assets/userfiles/sys_eb538c1c-65ff-4e82-8e6a-a1ef01127fed/files/ZIP/Fall-train.rar
# !unrar x $HOST_DATA_DIR/Fall-train.rar $HOST_DATA_DIR/train_raw
# !wget -P $HOST_DATA_DIR https://best.sjtu.edu.cn/Assets/userfiles/sys_eb538c1c-65ff-4e82-8e6a-a1ef01127fed/files/ZIP/Fall-test.rar
# !unrar x $HOST_DATA_DIR/Fall-test.rar $HOST_DATA_DIR/test_raw
# !wget -P $HOST_DATA_DIR https://best.sjtu.edu.cn/Assets/userfiles/sys_eb538c1c-65ff-4e82-8e6a-a1ef01127fed/files/ZIP/Squa-train.rar
# !unrar x $HOST_DATA_DIR/Squa-train.rar $HOST_DATA_DIR/train_raw
# !wget -P $HOST_DATA_DIR https://best.sjtu.edu.cn/Assets/userfiles/sys_eb538c1c-65ff-4e82-8e6a-a1ef01127fed/files/ZIP/Squa-test.rar
# !unrar x $HOST_DATA_DIR/Squa-test.rar $HOST_DATA_DIR/test_raw
# !wget -P $HOST_DATA_DIR https://best.sjtu.edu.cn/Assets/userfiles/sys_eb538c1c-65ff-4e82-8e6a-a1ef01127fed/files/ZIP/Sits-train.rar
# !unrar x $HOST_DATA_DIR/Sits-train.rar $HOST_DATA_DIR/train_raw
# !wget -P $HOST_DATA_DIR https://best.sjtu.edu.cn/Assets/userfiles/sys_eb538c1c-65ff-4e82-8e6a-a1ef01127fed/files/ZIP/Sits-test.rar
# !unrar x $HOST_DATA_DIR/Sits-test.rar $HOST_DATA_DIR/test_raw
# !wget -P $HOST_DATA_DIR https://best.sjtu.edu.cn/Assets/userfiles/sys_eb538c1c-65ff-4e82-8e6a-a1ef01127fed/files/ZIP/Walk-train.rar
# !unrar x $HOST_DATA_DIR/Walk-train.rar $HOST_DATA_DIR/train_raw
# !wget -P $HOST_DATA_DIR https://best.sjtu.edu.cn/Assets/userfiles/sys_eb538c1c-65ff-4e82-8e6a-a1ef01127fed/files/ZIP/Walk-test.rar
# !unrar x $HOST_DATA_DIR/Walk-test.rar $HOST_DATA_DIR/test_raw

`OPTIONAL` Run the process script for SHAD. 

`IMPORTANT NOTE`: to run the `process_SHAD.sh` generating optical flow, a Turing or Ampere above GPU is needed. You could run with `preprocess_SHAD_RGB.sh` to play with RGB frames only 

In [ ]:
# # ! cd tao_toolkit_recipes/tao_action_recognition/data_generation/ && ./preprocess_SHAD.sh $HOST_DATA_DIR/train_raw $HOST_DATA_DIR/train
# ! cd tao_toolkit_recipes/tao_action_recognition/data_generation/ && ./preprocess_SHAD_RGB.sh $HOST_DATA_DIR/train_raw $HOST_DATA_DIR/train

In [ ]:
# # ! cd tao_toolkit_recipes/tao_action_recognition/data_generation/ && ./preprocess_SHAD.sh $HOST_DATA_DIR/test_raw $HOST_DATA_DIR/test
# ! cd tao_toolkit_recipes/tao_action_recognition/data_generation/ && ./preprocess_SHAD_RGB.sh $HOST_DATA_DIR/test_raw $HOST_DATA_DIR/test

In [ ]:
# # verify
# !ls -l $HOST_DATA_DIR/train
# !ls -l $HOST_DATA_DIR/train/bend
# !ls -l $HOST_DATA_DIR/test
# !ls -l $HOST_DATA_DIR/test/bend

### 2.2 Download pretrained model from NGC

We will use NGC CLI to get the pre-trained models. For more details, go to https://ngc.nvidia.com and click the SETUP on the navigation bar.

In [9]:
# Installing NGC CLI on the local machine.
## Download and install
import os
%env CLI=ngccli_cat_linux.zip
!mkdir -p $HOST_RESULTS_DIR/ngccli

# Remove any previously existing CLI installations
!rm -rf $HOST_RESULTS_DIR/ngccli/*
!wget "https://ngc.nvidia.com/downloads/$CLI" -P $HOST_RESULTS_DIR/ngccli
!unzip -u "$HOST_RESULTS_DIR/ngccli/$CLI" -d $HOST_RESULTS_DIR/ngccli/
!rm $HOST_RESULTS_DIR/ngccli/*.zip 
os.environ["PATH"]="{}/ngccli:{}".format(os.getenv("HOST_RESULTS_DIR", ""), os.getenv("PATH", ""))

env: CLI=ngccli_cat_linux.zip
--2022-06-01 03:52:56--  https://ngc.nvidia.com/downloads/ngccli_cat_linux.zip
Resolving ngc.nvidia.com (ngc.nvidia.com)... 13.225.121.94, 13.225.121.41, 13.225.121.127, ...
Connecting to ngc.nvidia.com (ngc.nvidia.com)|13.225.121.94|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32617431 (31M) [application/zip]
Saving to: ‘/home/sandia/tao/results/ngccli/ngccli_cat_linux.zip’

ngccli_cat_linux.zi 100%[===================>]  31.11M  14.6MB/s    in 2.1s    

2022-06-01 03:52:59 (14.6 MB/s) - ‘/home/sandia/tao/results/ngccli/ngccli_cat_linux.zip’ saved [32617431/32617431]

Archive:  /home/sandia/tao/results/ngccli/ngccli_cat_linux.zip
  inflating: /home/sandia/tao/results/ngccli/ngc  
 extracting: /home/sandia/tao/results/ngccli/ngc.md5  


In [5]:
%cd results/ngccli
!./ngc registry model list nvidia/tao/actionrecognitionnet:*

/home/sandia/tao/results/ngccli
+-------+-------+-------+-------+-------+-------+-------+-------+-------+
| Versi | Accur | Epoch | Batch | GPU   | Memor | File  | Statu | Creat |
| on    | acy   | s     | Size  | Model | y Foo | Size  | s     | ed    |
|       |       |       |       |       | tprin |       |       | Date  |
|       |       |       |       |       | t     |       |       |       |
+-------+-------+-------+-------+-------+-------+-------+-------+-------+
| train | 88.0  | 120   | 1     | V100  | 850.1 | 850.1 | UPLOA | May   |
| able_ |       |       |       |       |       | 1 MB  | D_COM | 17,   |
| v2.0  |       |       |       |       |       |       | PLETE | 2022  |
| train | 88.0  | 120   | 1     | V100  | 426.2 | 426.1 | UPLOA | Nov   |
| able_ |       |       |       |       |       | 6 MB  | D_COM | 23,   |
| v1.0  |       |       |       |       |       |       | PLETE | 2021  |
| deplo | 88.0  | 120   | 1     | V100  | 339.8 | 339.7 | UPLOA | May   |
| yabl

In [11]:
!mkdir -p $HOST_RESULTS_DIR/pretrained

In [13]:
# Pull pretrained model from NGC 
!ngc registry model download-version "nvidia/tao/actionrecognitionnet:trainable_v1.0" --dest $HOST_RESULTS_DIR/pretrained

Downloaded 363.87 MB in 1m 19s, Download speed: 4.6 MB/s                
----------------------------------------------------
Transfer id: actionrecognitionnet_vtrainable_v1.0 Download status: Completed.
Downloaded local path: /home/sandia/tao/results/pretrained/actionrecognitionnet_vtrainable_v1.0
Total files downloaded: 2 
Total downloaded size: 363.87 MB
Started at: 2022-06-01 03:55:53.952336
Completed at: 2022-06-01 03:57:13.067109
Duration taken: 1m 19s
----------------------------------------------------


In [6]:
print("Check that model is downloaded into dir.")
!ls -l $HOST_RESULTS_DIR/pretrained/actionrecognitionnet_vtrainable_v1.0

Check that model is downloaded into dir.
total 436400
-rw------- 1 sandia sandia 114846245  6월  1 03:57 resnet18_2d_rgb_hmdb5_32.tlt
-rw------- 1 sandia sandia 332019621  6월  1 03:57 resnet18_3d_rgb_hmdb5_32.tlt


## 3. Provide training specification <a class="anchor" id="head-2"></a>

We provide specification files to configure the training parameters including:

* model_config: configure the model setting
    * model_type: type of model, rgb/of/joint
    * backbone: resnet18/34/50/101/152 
    * rgb_seq_length: length of RGB input sequence
    * input_type: 2d/3d
    * sample_strategy: consecutive
    * dropout_ratio: probability to drop the hidden units
* train_config: configure the training hyperparameters
    * optim_config
    * epochs
    * checkpoint_interval
* dataset_config: configure the dataset and augmentation methods
    * train_dataset_dir
    * val_dataset_dir
    * label_map: map the class label to id
    * output_shape
    * batch_size
    * workers: number of workers to do data loading
    * clips_per_video: number of clips to be sampled from single video
    * augmentation_config

Please refer to the TAO documentation about ActionRecognitionNet to get all the parameters that are configurable.

In [7]:
!cat $HOST_SPECS_DIR/train_rgb_3d_finetune.yaml

output_dir: /results/rgb_3d_ptm
encryption_key: nvidia_tao

## Model Configuration
model_config:                                                                        
   model_type: rgb                                                                     
   input_type: "3d"                                                                     
   backbone: resnet18                                                                 
   rgb_seq_length: 32  ## Change from 3 to 32 frame sequence                                                                
   rgb_pretrained_num_classes: 5                                                   
   sample_strategy: consecutive                                                        
   sample_rate: 1
   dropout_ratio: 0.0

# Training Hyperparameter configuration
train_config: 
   optim:   
      lr: 0.001  
      momentum: 0.9   
      weight_decay: 0.0001   
      lr_scheduler: MultiStep   
      lr_steps: [5, 15, 25]   
      lr_decay: 0.1 
   ep

## 4. Run TAO training <a class="anchor" id="head-3"></a>
* Provide the sample spec file and the output directory location for models
* WARNING: training will take several hours or one day to complete

In [8]:
# NOTE: The following paths are set from the perspective of the TAO Docker.

# The data is saved here
%env DATA_DIR = /data
%env SPECS_DIR = /specs
%env RESULTS_DIR = /results

env: DATA_DIR=/data
env: SPECS_DIR=/specs
env: RESULTS_DIR=/results


### 4.1 Train 3D model:

We provide pretrained RGB-only model trained on HMDB5 dataset. With the pretrained model, we can even get better accuracy with less epochs.

`KNOWN ISSUE`: 
- 1) The training log will be corrupted by pytorch warning in the notebook:

     `[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)` 
     
     To see the full log in std out, please run the command in terminal. 
- 2) "=" in the checkpoint file name should removed before using the checkpoint in command.

In [11]:
print("Train RGB only model with PTM")
!tao action_recognition train \
                  -e $SPECS_DIR/train_rgb_3d_finetune.yaml \
                  -r $RESULTS_DIR/rgb_3d_ptm/2nd \
                  -k $KEY \
                  model_config.rgb_pretrained_model_path=$RESULTS_DIR/pretrained/actionrecognitionnet_vtrainable_v1.0/resnet18_3d_rgb_hmdb5_32.tlt  \
                  model_config.rgb_pretrained_num_classes=5

Train RGB only model with PTM
/bin/bash: tao: command not found


In [24]:
print('Encrypted checkpoints:')
print('---------------------')
!ls -ltrh $HOST_RESULTS_DIR/rgb_3d_ptm

Encrypted checkpoints:
---------------------
total 6.2G
drwxr-xr-x 3 root root 4.0K  6월  1 04:32  lightning_logs
-rw-r--r-- 1 root root 317M  6월  1 04:33 'ar_model_epoch=00-val_loss=0.94.tlt'
-rw-r--r-- 1 root root 317M  6월  1 04:34 'ar_model_epoch=01-val_loss=0.82.tlt'
-rw-r--r-- 1 root root 317M  6월  1 04:35 'ar_model_epoch=02-val_loss=0.77.tlt'
-rw-r--r-- 1 root root 317M  6월  1 04:37 'ar_model_epoch=03-val_loss=0.75.tlt'
-rw-r--r-- 1 root root 317M  6월  1 04:38 'ar_model_epoch=04-val_loss=0.75.tlt'
-rw-r--r-- 1 root root 317M  6월  1 04:39 'ar_model_epoch=05-val_loss=0.73.tlt'
-rw-r--r-- 1 root root 317M  6월  1 04:40 'ar_model_epoch=06-val_loss=0.76.tlt'
-rw-r--r-- 1 root root 317M  6월  1 04:41 'ar_model_epoch=07-val_loss=0.72.tlt'
-rw-r--r-- 1 root root 317M  6월  1 04:42 'ar_model_epoch=08-val_loss=0.73.tlt'
-rw-r--r-- 1 root root 317M  6월  1 04:43 'ar_model_epoch=09-val_loss=0.74.tlt'
-rw-r--r-- 1 root root 317M  6월  1 04:44 'ar_model_epoch=10-val_loss=0.76.tlt'
-rw-r--r-- 1 root 

In [28]:
print('Rename a model: Note that the training is not deterministic, so you may change the model name accordingly.')
print('---------------------')
# NOTE: The following command may require `sudo`. You can run the command outside the notebook.
!mv $HOST_RESULTS_DIR/rgb_3d_ptm/ar_model_epoch=16-val_loss=0.72.tlt $HOST_RESULTS_DIR/rgb_3d_ptm/rgb_only_model.tlt 
!ls -ltrh $HOST_RESULTS_DIR/rgb_3d_ptm/rgb_only_model.tlt

Rename a model: Note that the training is not deterministic, so you may change the model name accordingly.
---------------------
[sudo] password for sandia: 
ls: cannot access '/home/sandia/tao/results/rgb_3d_ptm/rgb_only_model.tlt': No such file or directory


### `OPTIONAL` 4.2 Train 2D model

`Important Note` The following cells are using SHAD dataset. 

Firstly, we will train a 2D RGB-only model from scratch

In [ ]:
# print("Train RGB only model from scratch")
# !tao action_recognition train \
#                   -e $SPECS_DIR/train_rgb_2d.yaml \
#                   -r $RESULTS_DIR/rgb_2d \
#                   -k $KEY \
#                   dataset_config.train_dataset_dir=$DATA_DIR/train \
#                   dataset_config.val_dataset_dir=$DATA_DIR/test

In [ ]:
# print("To resume training from a checkpoint, set the resume_training_checkpoint_path option to be the .tlt you want to resume from")
# print("remember to remove the `=` in the checkpoint's file name")
# !tao action_recognition train \
#                   -e $SPECS_DIR/train_rgb_2d.yaml \
#                   -r $RESULTS_DIR/rgb_2d \
#                   -k $KEY \
#                   resume_training_checkpoint_path=

In [ ]:
# print('Encrypted checkpoints:')
# print('---------------------')
# !ls -ltrh $HOST_RESULTS_DIR/rgb_2d

In [ ]:
# print('Rename a model: ')
# print('---------------------')
# !echo <passwd> | sudo -S mv $HOST_RESULTS_DIR/rgb_2d/ar_model_epoch=21-val_loss=0.88.tlt $HOST_RESULTS_DIR/rgb_2d/rgb_only_model.tlt 
# !ls -ltrh $HOST_RESULTS_DIR/rgb_2d/rgb_only_model.tlt

Next, we will train a 2D optical flow only model from scratch

In [ ]:
# print("Train optical flow only model from scratch")
# !tao action_recognition train \
#                   -e $SPECS_DIR/train_of_2d.yaml \
#                   -r $RESULTS_DIR/of_2d \
#                   -k $KEY \
#                   dataset_config.train_dataset_dir=$DATA_DIR/train \
#                   dataset_config.val_dataset_dir=$DATA_DIR/test

In [ ]:
# print('Encrypted checkpoints:')
# print('---------------------')
# !ls -ltrh $HOST_RESULTS_DIR/of_2d

In [ ]:
# print('Rename a model: ')
# print('---------------------')
# !echo <passwd> | sudo -S mv $HOST_RESULTS_DIR/of_2d/ar_model_epoch=29-val_loss=0.94.tlt $HOST_RESULTS_DIR/of_2d/of_only_model.tlt 
# !ls -ltrh $HOST_RESULTS_DIR/of_2d/of_only_model.tlt

Finally, we will train a 2D joint model which consumed both RGB frames and optical flow frames based on two pretrained single stream model

In [ ]:
# print("Train joint model based on RGB and OF model")
# !tao action_recognition train \
#                   -e $SPECS_DIR/train_joint_2d.yaml \
#                   -r $RESULTS_DIR/joint_2d \
#                   -k $KEY \
#                   model_config.rgb_pretrained_model_path=$RESULTS_DIR/rgb_2d/rgb_only_model.tlt \
#                   model_config.of_pretrained_model_path=$RESULTS_DIR/of_2d/of_only_model.tlt \
#                   dataset_config.train_dataset_dir=$DATA_DIR/train \
#                   dataset_config.val_dataset_dir=$DATA_DIR/test

In [ ]:
# print('Encrypted checkpoints:')
# print('---------------------')
# !ls -ltrh $HOST_RESULTS_DIR/joint_2d

In [ ]:
# print('Rename a model: ')
# print('---------------------')
# !echo <passwd> | sudo -S mv $HOST_RESULTS_DIR/joint_2d/ar_model_epoch=16-val_loss=0.60.tlt $HOST_RESULTS_DIR/joint_2d/joint_model.tlt 
# !ls -ltrh $HOST_RESULTS_DIR/joint_2d/joint_model.tlt

## 5. Evaluate trained models <a class="anchor" id="head-4"></a>

We provide two different sample strategy to evaluate the pretrained model on video clips.

* `center` mode: pick up the middle frames of a sequence to do inference. For example, if the model requires 32 frames as input and a video clip has 128 frames, then we will choose the frames from index 48 to index 79 to do the inference. 
* `conv` mode: convolutionly sample 10 sequences out of a single video and do inference. The final results are averaged.

Evaluate RGB model trained with PTM

In [35]:
!tao action_recognition evaluate \
                    -e $SPECS_DIR/evaluate_rgb.yaml \
                    -k $KEY \
                    model=$RESULTS_DIR/rgb_3d_ptm/rgb_only_model.tlt  \
                    batch_size=1 \
                    test_dataset_dir=$DATA_DIR/test \
                    video_eval_mode=center

2022-06-01 06:33:36,731 [INFO] root: Registry: ['nvcr.io']
2022-06-01 06:33:36,879 [INFO] tlt.components.instance_handler.local_instance: Running command in container: nvcr.io/nvidia/tao/tao-toolkit-pyt:v3.21.11-py3
2022-06-01 06:33:36,979 [WARNING] tlt.components.docker_handler.docker_handler: 
Docker will run the commands as root. If you would like to retain your
local host permissions, please add the "user":"UID:GID" in the
DockerOptions portion of the "/home/sandia/.tao_mounts.json" file. You can obtain your
users UID and GID by using the "id -u" and "id -g" commands on the
terminal.
/home/jenkins/agent/workspace/tlt-pytorch-main-nightly/cv/action_recognition/scripts/evaluate.py:154: UserWarning: 
'evaluate_rgb.yaml' is validated against ConfigStore schema with the same name.
This behavior is deprecated in Hydra 1.1 and will be removed in Hydra 1.2.
See https://hydra.cc/docs/next/upgrades/1.0_to_1.1/automatic_schema_matching for migration instructions.
ResNet3d(
  (conv1): Conv3d(3

## 6. Inferences <a class="anchor" id="head-5"></a>
In this section, we run the action recognition inference tool to generate inferences with the trained RGB models and print the results. 

There are also two modes for inference just like evaluation: `center` mode and `conv` mode. And the final output will show each input sequence label in the videos like:
`[video_sample_path] [labels list for sequences in the video sample]`

In [38]:
!tao action_recognition inference \
                    -e $SPECS_DIR/infer_rgb.yaml \
                    -k $KEY \
                    model=$RESULTS_DIR/rgb_3d_ptm/rgb_only_model.tlt \
                    inference_dataset_dir=$DATA_DIR/test/pullup \
                    video_inf_mode=center

2022-06-01 06:50:00,072 [INFO] root: Registry: ['nvcr.io']
2022-06-01 06:50:00,214 [INFO] tlt.components.instance_handler.local_instance: Running command in container: nvcr.io/nvidia/tao/tao-toolkit-pyt:v3.21.11-py3
2022-06-01 06:50:00,309 [WARNING] tlt.components.docker_handler.docker_handler: 
Docker will run the commands as root. If you would like to retain your
local host permissions, please add the "user":"UID:GID" in the
DockerOptions portion of the "/home/sandia/.tao_mounts.json" file. You can obtain your
users UID and GID by using the "id -u" and "id -g" commands on the
terminal.
[NeMo W 2022-05-31 21:50:06 nemo_logging:349] /home/jenkins/agent/workspace/tlt-pytorch-main-nightly/cv/action_recognition/scripts/inference.py:88: UserWarning: 
    'infer_rgb.yaml' is validated against ConfigStore schema with the same name.
    This behavior is deprecated in Hydra 1.1 and will be removed in Hydra 1.2.
    See https://hydra.cc/docs/next/upgrades/1.0_to_1.1/automatic_schema_matching fo

2022-06-01 06:50:14,595 [INFO] tlt.components.docker_handler.docker_handler: Stopping container.


## 7. Deploy! <a class="anchor" id="head-6"></a>

In [39]:
!mkdir -p $HOST_RESULTS_DIR/export

In [41]:
# Export the RGB model to encrypted ONNX model
!tao action_recognition export \
                   -e $SPECS_DIR/export_rgb.yaml \
                   -k $KEY \
                   model=$RESULTS_DIR/rgb_3d_ptm/rgb_only_model.tlt\
                   output_file=$RESULTS_DIR/export/rgb_resnet18_3.etlt

2022-06-01 06:55:17,486 [INFO] root: Registry: ['nvcr.io']
2022-06-01 06:55:17,647 [INFO] tlt.components.instance_handler.local_instance: Running command in container: nvcr.io/nvidia/tao/tao-toolkit-pyt:v3.21.11-py3
2022-06-01 06:55:17,725 [WARNING] tlt.components.docker_handler.docker_handler: 
Docker will run the commands as root. If you would like to retain your
local host permissions, please add the "user":"UID:GID" in the
DockerOptions portion of the "/home/sandia/.tao_mounts.json" file. You can obtain your
users UID and GID by using the "id -u" and "id -g" commands on the
terminal.
[NeMo W 2022-05-31 21:55:23 nemo_logging:349] /home/jenkins/agent/workspace/tlt-pytorch-main-nightly/cv/action_recognition/scripts/export.py:155: UserWarning: 
    'export_rgb.yaml' is validated against ConfigStore schema with the same name.
    This behavior is deprecated in Hydra 1.1 and will be removed in Hydra 1.2.
    See https://hydra.cc/docs/next/upgrades/1.0_to_1.1/automatic_schema_matching for

2022-06-01 06:55:34,107 [INFO] tlt.components.docker_handler.docker_handler: Stopping container.


In [42]:
print('Exported model:')
print('------------')
!ls -lth $HOST_RESULTS_DIR/export

Exported model:
------------
total 127M
-rw-r--r-- 1 root root 127M  6월  1 06:55 rgb_resnet18_3.etlt


This notebook has come to an end. You may continue by deploying this model to [DeepStream](https://docs.nvidia.com/metropolis/deepstream/dev-guide/text/DS_3D_Action.html)